# Import Libraries

In [3]:
import numpy as np
import os
import shutil 
import cv2
import datetime
import glob
import json
from PIL import Image

# Convert RGB Images to grayscale format and saving them to another folder

In [19]:
destination_path = 'C:\For programming\Computer vision\dataset_grayscale'
images_path = 'C:\For programming\Computer vision\YOLO_Dataset'


def save_pil_gray(images_path, destination_path):
    if not os.path.exists(destination_path):
        os.makedirs(destination_path) 
        
    for image in glob.glob(images_path + '\*.jpg'):
        without = image.split('\\')[-1]
        gray = Image.open(image).convert('L')
        gray.save(os.path.join(destination_path, without))
        
save_pil_gray(images_path=images_path, destination_path=destination_path)

# Creating folder with labels

In [17]:
new_folder = '/home/sergey/json_dataset'   

os.makedirs(new_folder)
os.makedirs(new_folder + '/train/labels')
os.makedirs(new_folder + '/train/images')
os.makedirs(new_folder + '/val/labels')
os.makedirs(new_folder + '/val/images')
os.makedirs(new_folder + '/test/labels')
os.makedirs(new_folder + '/test/images')

# os.makedirs(new_folder + '/val/labels')
# os.makedirs(new_folder + '/test/labels')

In [60]:
images = glob.glob('/home/sergey/dataset_grayscale/*.jpg')

for img_path in images:
    if np.random.uniform(0,1) < 0.7:
        shutil.copy(img_path, new_folder + '/train/images')
    else:
        shutil.copy(img_path, new_folder + '/val/images')   

In [68]:
val_images = glob.glob('/home/sergey/json_dataset/val/images/*.jpg')

for val_img in val_images:
    if np.random.uniform(0,1) < 0.66:
        continue
    else:
        shutil.move(val_img, new_folder + '/test/images')   

In [11]:
val_images = glob.glob('/home/sergey/json_dataset/val/images/*.jpg')
train_images = glob.glob('/home/sergey/json_dataset/train/images/*.jpg')
test_images = glob.glob('/home/sergey/json_dataset/test/images/*.jpg')

train_dct = {}
val_dct = {}
test_dct = {}

for i in train_images:
    name = i.split('/')[-1]
    without = name.split('.')[0]
    train_dct[without] = 0

display(len(train_images))
display(len(train_dct))

for j in val_images:
    name = j.split('/')[-1]
    without = name.split('.')[0]
    val_dct[without] = 0

display(len(val_images))
display(len(val_dct))


for z in test_images:
    name = z.split('/')[-1]
    without = name.split('.')[0]
    test_dct[without] = 0

display(len(test_images))
display(len(test_dct))

5483

5483

1484

1484

781

781

In [18]:
labels = glob.glob('/home/sergey/dataset_grayscale/*.txt')

for txt_file in labels:

    name = txt_file.split('/')[-1]
    without = name.split('.')[0]
    
    if without in train_dct:
        shutil.copy(txt_file, new_folder + '/train/labels')
        
    if without in val_dct:
        shutil.copy(txt_file, new_folder + '/val/labels')
        
    if without in test_dct:
        shutil.copy(txt_file, new_folder + '/test/labels')
        

# From txt darknet to json coco

In [30]:
yolo_format_classes_path = r'/home/sergey/darknet_weights/obj.names'

def yolo_to_json(directory_images, directory_labels):
    write_json_context = {}
    categories = []
    
    with open(yolo_format_classes_path, 'r') as f:
        lines = f.read().splitlines()

    
    for ind, label in enumerate(lines):
        categories.append({'id': ind, 'name': label,
                           'supercategory': label})

    

    write_json_context['info'] = {"year": 2021,
            "version": "",
            "description": "",
            "contributor": "",
            "url": "",
            "date_created": f'{datetime.datetime.now()}'}

    write_json_context['licenses'] = [{'id': 1, 
                                       'name': None, 
                                       'url': None}]
    write_json_context['categories'] = categories
    write_json_context['images'] = []
    write_json_context['annotations'] = []
    
    file_number = 1
    num_bboxes = 1
    
    for file in os.listdir(directory_images):
        filename = os.fsdecode(file)
        if filename.endswith(".jpg"):
            
            img_path = (os.path.join(directory_images, filename))
            base=os.path.basename(img_path)
            file_name_without_ext = os.path.splitext(base)[0] 
            yolo_annotation_path  = os.path.join(directory_labels, file_name_without_ext+ "." + 'txt')
            img_name = os.path.basename(img_path) 
            
            img_context = {}
            height,width = cv2.imread(img_path).shape[:2]
            img_context['date_captured'] = f'{datetime.datetime.now()}'
            img_context['id'] = file_number # image id
            img_context['file_name'] = img_name
            img_context['height'] = height
            img_context['width'] = width
            img_context['license'] = 1
            img_context['coco_url'] =''
            img_context['flickr_url'] = ''
            write_json_context['images'].append(img_context)

            with open(yolo_annotation_path,'r') as f2:
                lines2 = f2.readlines() 

            for i,line in enumerate(lines2): 
                line = line.split(' ')
                bbox_dict = {}
                class_id, x_yolo,y_yolo,width_yolo,height_yolo= line[0:]
                x_yolo,y_yolo,width_yolo,height_yolo,class_id= float(x_yolo),float(y_yolo),float(width_yolo),float(height_yolo),int(class_id)
                bbox_dict['id'] = num_bboxes
                bbox_dict['image_id'] = file_number
                bbox_dict['category_id'] = class_id
                bbox_dict['iscrowd'] = 0 
                h,w = round(abs(height_yolo*height)),round(abs(width_yolo*width))
                bbox_dict['area']  = h * w
                x_coco = round(x_yolo*width -(w/2))
                y_coco = round(y_yolo*height -(h/2))
                if x_coco <0: 
                    x_coco = 1
                if y_coco <0: 
                    y_coco = 1
                bbox_dict['bbox'] = [x_coco,y_coco,w,h]
                bbox_dict['segmentation'] = []
                write_json_context['annotations'].append(bbox_dict)
                num_bboxes+=1

            file_number = file_number+1
            continue
        else:
            continue
        
    return write_json_context

In [31]:
labels_dirs = [new_folder + '/train/labels', new_folder + '/val/labels', new_folder + '/test/labels']
images_dirs = [new_folder + '/train/images', new_folder + '/val/images', new_folder + '/test/images']
zipped = list(zip(images_dirs, labels_dirs))
zipped

[('/home/sergey/json_dataset/train/images',
  '/home/sergey/json_dataset/train/labels'),
 ('/home/sergey/json_dataset/val/images',
  '/home/sergey/json_dataset/val/labels'),
 ('/home/sergey/json_dataset/test/images',
  '/home/sergey/json_dataset/test/labels')]

In [33]:
ans = yolo_to_json(*zipped[2])
ans

{'info': {'year': 2021,
  'version': '',
  'description': '',
  'contributor': '',
  'url': '',
  'date_created': '2021-08-18 18:37:07.697305'},
 'licenses': [{'id': 1, 'name': None, 'url': None}],
 'categories': [{'id': 0, 'name': 'Cone', 'supercategory': 'Cone'}],
 'images': [{'file_name': 'vid_5_frame_2272.jpg',
   'height': 720,
   'width': 1280,
   'date_captured': '2021-08-18 18:37:07.704435',
   'id': 1,
   'license': 1,
   'coco_url': '',
   'flickr_url': ''},
  {'file_name': 'vid_3_frame_38536.jpg',
   'height': 720,
   'width': 1280,
   'date_captured': '2021-08-18 18:37:07.712090',
   'id': 2,
   'license': 1,
   'coco_url': '',
   'flickr_url': ''},
  {'file_name': 'vid_42_frame_329.jpg',
   'height': 1536,
   'width': 2048,
   'date_captured': '2021-08-18 18:37:07.734975',
   'id': 3,
   'license': 1,
   'coco_url': '',
   'flickr_url': ''},
  {'file_name': 'vid_64_frame_390.jpg',
   'height': 592,
   'width': 800,
   'date_captured': '2021-08-18 18:37:07.739413',
   'id':

In [32]:
for images, labels in zipped:
    ans = yolo_to_json(directory_images=images, directory_labels=labels)
    with open(images + '/_annotations.coco.json', 'w') as file:
        json.dump(ans, file)